In [1]:
import requests
import json
import warnings
import pandas as pd
from datetime import datetime, timedelta, date
from pprint import pprint

# OBTENCIÓN DE DATOS DE INTERCAMBIO 

In [2]:
lang = input('¿En español (es) o en inglés (en)?')
restaDia = float(input("¿Cuantos días atrás?: ______________"))
input_año = int(input("¿Qué año?: ______________"))

In [3]:
headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Host' : 'apidatos.ree.es'
}

In [6]:
def intercambio_datos(lang, restaDia, input_año):

    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')
    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"


    endpoint = f"https://apidatos.ree.es/{lang}/datos/intercambios/todas-fronteras-programados?{query}"
    response = requests.get(url = endpoint, headers = headers)
    data = response.json()


    lista_nombres = list()
    lista_tipos = list()
    lista_valores = list()
    lista_porcentajes = list()
    lista_fechas = list()
   


    for dato in data['included']: 
        for info in dato['attributes']['content']:
            tipo = info['type']

            nombre = info['groupId']


            for i in info['attributes']['values']:
                valor = i['value']

                porcentaje = i['percentage']

                fecha = i['datetime']
                fecha = pd.to_datetime(fecha)
                

                fecha =  fecha.strftime("%d/%m/%Y")

                
                lista_nombres.append(nombre)
                lista_tipos.append(tipo)
                lista_valores.append(valor)
                lista_porcentajes.append(porcentaje)
                lista_fechas.append(fecha)
               


    df_intercambio = pd.DataFrame()
    df_intercambio['nombre']               = lista_nombres
    df_intercambio['tipo de intercambio']      = lista_tipos
    df_intercambio['Valores']              = lista_valores
    df_intercambio["Porcentaje"]           = lista_porcentajes
    df_intercambio["Fecha actualización"]  = lista_fechas
             


    return df_intercambio 

df_intercambio = intercambio_datos(lang, restaDia, input_año)

df_intercambio

,nombre,tipo de intercambio,Valores,Porcentaje,Fecha actualización
0,Francia,Exportación,-19961.300,0.330618,08/09/2023
1,Francia,Exportación,-15923.134,0.243451,09/09/2023
2,Francia,Exportación,-21133.600,0.315685,10/09/2023
3,Francia,Exportación,-11122.500,0.202408,11/09/2023
4,Francia,Exportación,-18856.900,0.299353,12/09/2023
...,...,...,...,...,...
326,Andorra,saldo,-434.000,1.000000,04/10/2023
327,Andorra,saldo,-410.000,1.000000,05/10/2023
328,Andorra,saldo,-437.000,1.000000,06/10/2023
329,Andorra,saldo,-399.000,1.000000,07/10/2023


In [7]:
df_intercambio.to_csv('intercambio_electrico.csv')